In [73]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/a-advanced-regression-techniques/sample_submission.csv
/kaggle/input/a-advanced-regression-techniques/data_description.txt
/kaggle/input/a-advanced-regression-techniques/train.csv
/kaggle/input/a-advanced-regression-techniques/test.csv


# Intro Housing Price Regression Walkthrough 

I touch on a few things in the **notebook**:
1. Basic data cleaning and feature exploration
2. Exploratory data analysis (Answering questions we have of the data)
3. Basic Data Engineering (Creating a pipeline for tain and test sets)
4. Model Experimentation and parameter tuning (Linear Regression, Random Forest, XGBoost, MLP)
5. Feature Engineering 
6. Ensembling 
7. Submitting to the Competition

In [74]:
import warnings

# Ignore all warnings
warnings.filterwarnings("ignore")

# Your code goes here

# Reset warnings to the default behavior if needed
warnings.resetwarnings()


In [75]:
#import relevant packages 
import pandas as pd 
import numpy as np 
import plotly.graph_objects as go
import plotly.express as px
import scipy.stats as stats
from IPython.display import display, HTML
pd.set_option('display.max_columns', None)
from sklearn.preprocessing import FunctionTransformer
from sklearn.compose import make_column_transformer , ColumnTransformer
from sklearn.pipeline import make_pipeline , Pipeline
from scipy.stats import skew
from sklearn.preprocessing import MinMaxScaler , StandardScaler
from sklearn.impute import SimpleImputer , KNNImputer
from sklearn.preprocessing import OneHotEncoder , LabelEncoder
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso
from sklearn.model_selection import GridSearchCV



# Basic Data Exploration
1. Import the data
2. Look at summary statisitcs 
3. Evaluate Null Values

In [76]:
df = pd.read_csv("/kaggle/input/a-advanced-regression-techniques/train.csv")

In [77]:
df.shape

(1460, 81)

In [78]:
# Extract numeric and categorical columns
num_cols = df.select_dtypes(include=[np.number]).columns
cat_cols = df.select_dtypes(exclude=[np.number]).columns


In [79]:
# Summary statistics for numerical features
df[num_cols].describe()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
count,1460.000000,1460.000000,1201.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1452.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1379.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000
mean,730.500000,56.897260,70.049958,10516.828082,6.099315,5.575342,1971.267808,1984.865753,103.685262,443.639726,46.549315,567.240411,1057.429452,1162.626712,346.992466,5.844521,1515.463699,0.425342,0.057534,1.565068,0.382877,2.866438,1.046575,6.517808,0.613014,1978.506164,1.767123,472.980137,94.244521,46.660274,21.954110,3.409589,15.060959,2.758904,43.489041,6.321918,2007.815753,180921.195890
std,421.610009,42.300571,24.284752,9981.264932,1.382997,1.112799,30.202904,20.645407,181.066207,456.098091,161.319273,441.866955,438.705324,386.587738,436.528436,48.623081,525.480383,0.518911,0.238753,0.550916,0.502885,0.815778,0.220338,1.625393,0.644666,24.689725,0.747315,213.804841,125.338794,66.256028,61.119149,29.317331,55.757415,40.177307,496.123024,2.703626,1.328095,79442.502883
min,1.000000,20.000000,21.000000,1300.000000,1.000000,1.000000,1872.000000,1950.000000,0.000000,0.000000,0.000000,0.000000,0.000000,334.000000,0.000000,0.000000,334.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,0.000000,1900.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2006.000000,34900.000000
25%,365.750000,20.000000,59.000000,7553.500000,5.000000,5.000000,1954.000000,1967.000000,0.000000,0.000000,0.000000,223.000000,795.750000,882.000000,0.000000,0.000000,1129.500000,0.000000,0.000000,1.000000,0.000000,2.000000,1.000000,5.000000,0.000000,1961.000000,1.000000,334.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,2007.000000,129975.000000
50%,730.500000,50.000000,69.000000,9478.500000,6.000000,5.000000,1973.000000,1994.000000,0.000000,383.500000,0.000000,477.500000,991.500000,1087.000000,0.000000,0.000000,1464.000000,0.000000,0.000000,2.000000,0.000000,3.000000,1.000000,6.000000,1.000000,1980.000000,2.000000,480.000000,0.000000,25.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,2008.000000,163000.000000
75%,1095.250000,70.000000,80.000000,11601.500000,7.000000,6.000000,2000.000000,2004.000000,166.000000,712.250000,0.000000,808.000000,1298.250000,1391.250000,728.000000,0.000000,1776.750000,1.000000,0.000000,2.000000,1.000000,3.000000,1.000000,7.000000,1.000000,2002.000000,2.000000,576.000000,168.000000,68.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000,2009.000000,214000.000000
max,1460.000000,190.000000,313.000000,215245.000000,10.000000,9.000000,2010.000000,2010.000000,1600.000000,5644.000000,1474.000000,2336.000000,6110.000000,4692.000000,2065.000000,572.000000,5642.000000,3.000000,2.000000,3.000000,2.000000,8.000000,3.000000,14.000000,3.000000,2010.000000,4.000000,1418.000000,857.000000,547.000000,552.000000,508.000000,480.000000,738.000000,15500.000000,12.000000,2010.000000,755000.000000


In [80]:
# Summary statistics for categorical features
df[cat_cols].describe()

,MSZoning,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinType2,Heating,HeatingQC,CentralAir,Electrical,KitchenQual,Functional,FireplaceQu,GarageType,GarageFinish,GarageQual,GarageCond,PavedDrive,PoolQC,Fence,MiscFeature,SaleType,SaleCondition
count,1460,1460,91,1460,1460,1460,1460,1460,1460,1460,1460,1460,1460,1460,1460,1460,1460,588,1460,1460,1460,1423,1423,1422,1423,1422,1460,1460,1460,1459,1460,1460,770,1379,1379,1379,1379,1460,7,281,54,1460,1460
unique,5,2,2,4,4,2,5,3,25,9,8,5,8,6,8,15,16,3,4,5,6,4,4,4,6,6,6,5,2,5,4,7,5,6,3,5,5,3,3,4,4,9,6
top,RL,Pave,Grvl,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Norm,Norm,1Fam,1Story,Gable,CompShg,VinylSd,VinylSd,BrkFace,TA,TA,PConc,TA,TA,No,Unf,Unf,GasA,Ex,Y,SBrkr,TA,Typ,Gd,Attchd,Unf,TA,TA,Y,Gd,MnPrv,Shed,WD,Normal
freq,1151,1454,50,925,1311,1459,1052,1382,225,1260,1445,1220,726,1141,1434,515,504,445,906,1282,647,649,1311,953,430,1256,1428,741,1365,1334,735,1360,380,870,605,1311,1326,1340,3,157,49,1267,1198


# Explore the dependent variable 
- Should it be normalized? 
- Normalize Dependent Vairable

In [81]:
import scipy.stats as stats

# Fit a normal distribution to the SalePrice data
mu, sigma = stats.norm.fit(df['SalePrice'])

# Create a histogram of the SalePrice column
hist_data = go.Histogram(x=df['SalePrice'], nbinsx=50, name="Histogram", opacity=0.75, histnorm='probability density', marker=dict(color='purple'))

# Calculate the normal distribution based on the fitted parameters
x_norm = np.linspace(df['SalePrice'].min(), df['SalePrice'].max(), 100)
y_norm = stats.norm.pdf(x_norm, mu, sigma)

# Create the normal distribution overlay
norm_data = go.Scatter(x=x_norm, y=y_norm, mode="lines", name=f"Normal dist. (μ={mu:.2f}, σ={sigma:.2f})", line=dict(color="green"))

# Combine the histogram and the overlay
fig = go.Figure(data=[hist_data, norm_data])

# Set the layout for the plot
fig.update_layout(
    title="SalePrice Distribution",
    xaxis_title="SalePrice",
    yaxis_title="Density",
    legend_title_text="Fitted Normal Distribution",
    plot_bgcolor='rgba(32, 32, 32, 1)',
    paper_bgcolor='rgba(32, 32, 32, 1)',
    font=dict(color='white')
)

# Create a Q-Q plot
qq_data = stats.probplot(df['SalePrice'], dist="norm")
qq_fig = px.scatter(x=qq_data[0][0], y=qq_data[0][1], labels={'x': 'Theoretical Quantiles', 'y': 'Ordered Values'}, color_discrete_sequence=["purple"])
qq_fig.update_layout(
    title="Q-Q plot",
    plot_bgcolor='rgba(32, 32, 32, 1)',
    paper_bgcolor='rgba(32, 32, 32, 1)',
    font=dict(color='white')
)

# Calculate the line of best fit
slope, intercept, r_value, p_value, std_err = stats.linregress(qq_data[0][0], qq_data[0][1])
line_x = np.array(qq_data[0][0])
line_y = intercept + slope * line_x

# Add the line of best fit to the Q-Q plot
line_data = go.Scatter(x=line_x, y=line_y, mode="lines", name="Normal Line", line=dict(color="green"))

# Update the Q-Q plot with the normal line
qq_fig.add_trace(line_data)

# Show the plots
fig.show()
qq_fig.show()

# What questions do we want to ask of the data?

1. Distribution of dwelling types and their relation to sale prices?
2. Does zoning impact sale price?
3. Does street and alley access types effect on sale price?
4. What is the Average sale price by property shape?
5. Is there a Correlation between Property Age and Sale Price
6. Is there a Correlation between Living Area and Sale Price
7. Does price change year to year?

In [82]:
# 1. Distribution of dwelling types and their relation to sale prices
dwelling_types = df['BldgType'].value_counts()
dwelling_prices = df.groupby('BldgType')['SalePrice'].mean()

# Format labels for the second graph
formatted_dwelling_prices = ['$' + f'{value:,.2f}' for value in dwelling_prices.values]

# Create bar charts
fig1 = go.Figure(data=[go.Bar(
    x=dwelling_types.index,
    y=dwelling_types.values,
    marker_color='rgb(76, 175, 80)',
    text=dwelling_types.values,
    textposition='outside',
    width=0.4,
    marker=dict(line=dict(width=2, color='rgba(0,0,0,1)'), opacity=1)
)])
fig1.update_layout(
    title='Distribution of Building Types',
    xaxis_title='Building Type',
    yaxis_title='Count',
    plot_bgcolor='rgba(34, 34, 34, 1)',
    paper_bgcolor='rgba(34, 34, 34, 1)',
    font=dict(color='white')
)

fig2 = go.Figure(data=[go.Bar(
    x=dwelling_prices.index,
    y=dwelling_prices.values,
    marker_color='rgb(156, 39, 176)',
    text=formatted_dwelling_prices,
    textposition='outside',
    width=0.4,
    marker=dict(line=dict(width=2, color='rgba(0,0,0,1)'), opacity=1)
)])
fig2.update_layout(
    title='Average Sale Price by Building Type',
    xaxis_title='Building Type',
    yaxis_title='Price',
    plot_bgcolor='rgba(34, 34, 34, 1)',
    paper_bgcolor='rgba(34, 34, 34, 1)',
    font=dict(color='white')
)

# Show the figures
fig1.show()
fig2.show()

In [83]:
# 2. Zoning impact on sale price
zoning_prices = df.groupby('MSZoning')['SalePrice'].mean()
fig3 = px.bar(x=zoning_prices.index, y=zoning_prices.values, title='Average Sale Price by Zoning',
              color_discrete_sequence=['purple', 'green'], text=zoning_prices.values,
              template='plotly_dark')

fig3.update_traces(texttemplate='$%{text:,.0f}', textposition='outside')
fig3.update_yaxes(title='Sale Price', tickprefix='$', tickformat=',')
fig3.update_xaxes(title='Zoning')
fig3.update_layout(uniformtext_minsize=8, uniformtext_mode='hide')

fig3.show()

In [84]:
# 3. Street and alley access types effect on sale price
street_prices = df.groupby('Street')['SalePrice'].mean()
alley_prices = df.groupby('Alley')['SalePrice'].mean()

# Street Prices
colors_street = np.where(street_prices.index == 'Pave', 'purple', 'green')
fig5 = px.bar(x=street_prices.index, y=street_prices.values, title='Average Sale Price by Street Type',
              template='plotly_dark', text=street_prices.values,
              color=colors_street, color_discrete_sequence=['purple', 'green'])

fig5.update_traces(texttemplate='$%{text:,.0f}', textposition='outside')
fig5.update_yaxes(title='Sale Price', tickprefix='$', tickformat=',')
fig5.update_xaxes(title='Street Type')
fig5.update_layout(showlegend=False)

# Alley Prices
colors_alley = np.where(alley_prices.index == 'Pave', 'purple', 'green')
fig6 = px.bar(x=alley_prices.index, y=alley_prices.values, title='Average Sale Price by Alley Type',
              template='plotly_dark', text=alley_prices.values,
              color=colors_alley, color_discrete_sequence=['purple', 'green'])

fig6.update_traces(texttemplate='$%{text:,.0f}', textposition='outside')
fig6.update_yaxes(title='Sale Price', tickprefix='$', tickformat=',')
fig6.update_xaxes(title='Alley Type')
fig6.update_layout(showlegend=False)

fig5.show()
fig6.show()

In [85]:
# 4. Average sale price by property shape
colors = px.colors.qualitative.Plotly

shape_prices = df.groupby('LotShape')['SalePrice'].mean()
contour_prices = df.groupby('LandContour')['SalePrice'].mean()
# Shape Prices
fig7 = px.bar(x=shape_prices.index, y=shape_prices.values, title='Average Sale Price by Property Shape',
              template='plotly_dark', text=shape_prices.values)

fig7.update_traces(marker_color=colors, texttemplate='$%{text:,.0f}', textposition='outside')
fig7.update_yaxes(title='Sale Price', tickprefix='$', tickformat=',')
fig7.update_xaxes(title='Property Shape')
fig7.update_layout(showlegend=False)

# Contour Prices
fig8 = px.bar(x=contour_prices.index, y=contour_prices.values, title='Average Sale Price by Property Contour',
              template='plotly_dark', text=contour_prices.values)

fig8.update_traces(marker_color=colors, texttemplate='$%{text:,.0f}', textposition='outside')
fig8.update_yaxes(title='Sale Price', tickprefix='$', tickformat=',')
fig8.update_xaxes(title='Property Contour')
fig8.update_layout(showlegend=False)

fig7.show()
fig8.show()

In [86]:
# 5. Calculate Property Age
df['PropertyAge'] = df['YrSold'] - df['YearBuilt']

# Calculate Correlation between Property Age and Sale Price
age_price_corr = df['PropertyAge'].corr(df['SalePrice'])
print(f'Correlation between Property Age and Sale Price: {age_price_corr}')

# Create a scatter plot to visualize the relationship between Property Age and Sale Price
fig9 = px.scatter(df, x='PropertyAge', y='SalePrice', title='Property Age vs Sale Price', color='PropertyAge', color_continuous_scale=px.colors.sequential.Purp)

fig9.update_layout(plot_bgcolor='rgb(30,30,30)', paper_bgcolor='rgb(30,30,30)', font=dict(color='white'))

fig9.show()

Correlation between Property Age and Sale Price: -0.523350417546816


In [87]:
# we will delete the col just we add to illustrate somtheing 
del df['PropertyAge']

In [88]:
# 6. Calculate Correlation between Living Area and Sale Price
living_area_price_corr = df['GrLivArea'].corr(df['SalePrice'])
print(f'Correlation between Living Area (above grade) and Sale Price: {living_area_price_corr}')

# Create a scatter plot to visualize the relationship between Living Area and Sale Price
fig10 = px.scatter(df, x='GrLivArea', y='SalePrice', title='Living Area (above grade) vs Sale Price', color='GrLivArea', color_continuous_scale=px.colors.sequential.Purp)

fig10.update_layout(plot_bgcolor='rgb(30,30,30)', paper_bgcolor='rgb(30,30,30)', font=dict(color='white'))

fig10.show()

Correlation between Living Area (above grade) and Sale Price: 0.7086244776126523


In [89]:
# 7. Box plot of price over the years
yearly_avg_sale_price = df.groupby('YrSold')['SalePrice'].mean()

fig13 = px.box(df, x='YrSold', y='SalePrice', title='Sale Price Trends Over the Years',
               points=False, color_discrete_sequence=['green'])

fig13.add_trace(px.line(x=yearly_avg_sale_price.index, y=yearly_avg_sale_price.values).data[0])

fig13.update_traces(line=dict(color='purple', width=4), selector=dict(type='scatter', mode='lines'))

for year, avg_price in yearly_avg_sale_price.items():
    fig13.add_annotation(
        x=year,
        y=avg_price,
        text=f"{avg_price:,.0f}",
        font=dict(color='white'),
        showarrow=False,
        bgcolor='rgba(128, 0, 128, 0.6)'
    )

fig13.update_layout(
    plot_bgcolor='rgb(30,30,30)',
    paper_bgcolor='rgb(30,30,30)',
    font=dict(color='white'),
    xaxis_title='Year Sold',
    yaxis_title='Sale Price'
)

fig13.show()

In [90]:

# Assuming `train` is your DataFrame
mean = df["SalePrice"].mean()
Q1 = np.percentile(df["SalePrice"], q=25, method='midpoint')  # Updated 'interpolation' to 'method'
Q3 = np.percentile(df["SalePrice"], q=75, method='midpoint')  # Updated 'interpolation' to 'method'
IQR = Q3 - Q1

upper_bound = Q3 + (3 * IQR)
lower_bound = Q1 - (3 * IQR)

# Control Chart
fig14 = px.box(df, x='YrSold', y='SalePrice', title='Sale Price Control Chart')

fig14.add_trace(go.Scatter(x=df.index, y=df['SalePrice'], mode='lines', name='SalePrice'))
fig14.add_trace(go.Scatter(x=df.index, y=[mean] * len(df), mode='lines', name='Mean', line=dict(color='red', dash='dash')))
fig14.add_trace(go.Scatter(x=df.index, y=[upper_bound] * len(df), mode='lines', name='Upper Limit', line=dict(color='green', dash='dash')))
fig14.add_trace(go.Scatter(x=df.index, y=[lower_bound] * len(df), mode='lines', name='Lower Limit', line=dict(color='green', dash='dash')))

fig14.update_layout(
    xaxis_title='Samples',
    yaxis_title='Sale Prices',
    plot_bgcolor='rgb(30,30,30)',
    paper_bgcolor='rgb(30,30,30)',
    font=dict(color='white')
)

fig14.show()


In [91]:
# Assuming `train_data` is your DataFrame and `num_cols` is a list of numerical columns
corr_matrix = df[num_cols].corr()

fig15 = go.Figure(data=go.Heatmap(
    z=corr_matrix.values,
    x=corr_matrix.columns,
    y=corr_matrix.columns,
    colorscale='Viridis',
    colorbar=dict(title='Correlation'),
))

fig15.update_layout(
    title='Correlation Heatmap',
    xaxis_title='Features',
    yaxis_title='Features',
    width=800,
    height=600,
    template='plotly_dark',  # Use dark theme
)

fig15.show()

## Creating Data Preprocessing Pipeline 

The data preprocessing pipeline consists of several key steps to prepare the dataset for machine learning models.

1. **Feature Engineering:**
   - Additional features are created, such as 'PropertyAge', 'TotalSF', 'TotalBath', 'HasRemodeled', 'Has2ndFloor', and 'HasGarage'.

2. **DropNullColumns:**
   - Columns with a null value percentage above a specified threshold (default is 60%) are dropped.

3. **Impute:**
   - Missing values are imputed for numerical features using KNNImputer.
   - Missing values for categorical features are imputed with a constant value ('missing').

4. **DropColumnsWithSameValues:**
   - Columns with a high percentage of identical values (above a specified threshold, default is 0.8) are dropped.

5. **DropCorrelationWithTarget:**
   - Highly correlated features with the target variable are identified and removed.

6. **OutliersClamping:**
   - Outliers in numerical columns are clamped within a specified range using the IQR method.

7. **SkewnessTransformer:**
   - Log transformation is applied to skewed numerical features.

8. **CustomDataPreprocessor:**
   - The final preprocessing step involves scaling numerical features, one-hot encoding nominal categorical features, and label binarization of ordinal categorical features using a custom transformer.

9. **Pipeline Composition:**
   - All the above steps are organized into a scikit-learn pipeline for seamless application to training and test datasets.

Note: Ensure that the provided ordinal_cats and nominal_cats variables are defined appropriately based on the dataset's categorical features.


In [92]:
def update(data):
    
    num_cols = data.select_dtypes(include=[np.number]).columns
    cat_cols = data.select_dtypes(exclude=[np.number]).columns
    return num_cols, cat_cols


###################################################################
# feature engineering functions 
def custom_features(df):
    df_out = df.copy()
    df_out['PropertyAge'] = df_out['YrSold'] - df_out['YearBuilt']
    df_out['TotalSF'] = df_out['TotalBsmtSF'] + df_out['1stFlrSF'] + df_out['2ndFlrSF']
    df_out['TotalBath'] = df_out['FullBath'] + 0.5 * df_out['HalfBath'] + df_out['BsmtFullBath'] + 0.5 * df['BsmtHalfBath']
    df_out['HasRemodeled'] = (df_out['YearRemodAdd'] != df_out['YearBuilt']).astype(object)
    df_out['Has2ndFloor'] = (df_out['2ndFlrSF'] > 0).astype(object)
    df_out['HasGarage'] = (df_out['GarageArea'] > 0).astype(object)
    df_out['YrSold_cat'] = df_out['YrSold'].astype(object)
    df_out['MoSold_cat'] = df_out['MoSold'].astype(object)
    df_out['YearBuilt_cat'] = df_out['YearBuilt'].astype(object)
    df_out['MSSubClass_cat'] = df_out['MSSubClass'].astype(object)
    
    return df_out

feature_engineering_transformer = FunctionTransformer(custom_features)

#####################################################################



class DropNullColumns(BaseEstimator, TransformerMixin):
    def __init__(self, threshold=58):
        self.threshold = threshold
        self.cols_to_drop = []

    def fit(self, X, y=None):
        # Get and store columns with null values
        self.cols_of_null = self.get_null_columns(X)
        self.cols_to_drop = [col for col in self.cols_of_null if col in X.columns]
        return self

    def transform(self, X):
        # Drop columns with null values
        X_copy = X.copy()
        X_copy = X_copy.drop(self.cols_to_drop, axis=1)
        return X_copy

    def get_null_columns(self, X):
        X_copy = X.copy()
        null_percentages = (X_copy.isnull().sum() / len(X_copy)) * 100
        cols_of_null = null_percentages[null_percentages > self.threshold].index
        return cols_of_null


#####################################################################################


class Impute(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        self.num_cols, self.cat_cols = update(X)
        
        # Fit numerical imputer
        self.imputer_num = KNNImputer()
        self.imputer_num.fit(X[self.num_cols])
        
        # Fit categorical imputer
        self.imputer_cat = SimpleImputer(strategy='constant', fill_value='missing')
        self.imputer_cat.fit(X[self.cat_cols])
        
        return self

    def transform(self, X):
        # Transform numerical columns
        X[self.num_cols] = self.imputer_num.transform(X[self.num_cols])
        
        # Transform categorical columns
        X[self.cat_cols] = self.imputer_cat.transform(X[self.cat_cols])
        
        return X


    
    
##############################################################################



class DropColumnsWithSameValues(BaseEstimator, TransformerMixin):
    def __init__(self, threshold=0.8):
        self.threshold = threshold
        self.cols_to_drop = []
    
    
    def fit(self, X, y=None):
        self.cols_to_drop = self.get_columns_with_same_values(X)
        self.cols_to_drop = [col for col in self.cols_to_drop if col in X.columns]
        return self


    def transform(self, X):
        X_copy = X.copy()
        X_copy = X_copy.drop(columns=self.cols_to_drop, axis=1)
        return X_copy

    def get_columns_with_same_values(self, data):
        cols_to_drop = []

        num_cols, cat_cols = update(data)
        # Check numerical columns
        for col in num_cols:
            unique_values = data.groupby(col).size().reset_index(name='count')
            condition = unique_values['count'] > self.threshold * unique_values['count'].sum()
            if len(unique_values[condition]) > 0:
                cols_to_drop.append(col)

        # Check categorical columns
        for col in cat_cols:
            unique_values = data.groupby(col).size().reset_index(name='count')
            condition = unique_values['count'] > self.threshold * unique_values['count'].sum()
            if len(unique_values[condition]) > 0:
                cols_to_drop.append(col)

        return cols_to_drop

#####################################################################################


# Function to remove highly correlated features
def remove_highly_correlated_features(data, threshold=0.7):
    # target variable
    response_variable = 'SalePrice'
    
    num_cols , cat_cols = update(data)
    
    # numerical columns without the target variable
    num_cols_without_response_variable = list(set(num_cols) - set([response_variable]))
    corr_matrix = data[num_cols_without_response_variable].corr()
    columns_to_drop = set()

    for row_idx in range(corr_matrix.shape[0]):
        for col_idx in range(row_idx + 1, corr_matrix.shape[0]):
            correlation_value = corr_matrix.iloc[row_idx, col_idx]

            if np.abs(correlation_value) > threshold:
                col_row = num_cols_without_response_variable[row_idx]
                col_col = num_cols_without_response_variable[col_idx]

                corr_row = np.abs(data[col_row].corr(data[response_variable]))
                corr_col = np.abs(data[col_col].corr(data[response_variable]))

                if corr_row > corr_col:
                    columns_to_drop.add(col_col)
                else:
                    columns_to_drop.add(col_row)

    
    
    return list(columns_to_drop)


correaltion_with_target = remove_highly_correlated_features(df) 



def drop_correlation_with_target(data, correaltion_with_target):
    data_copy = data.copy()  # Create a copy to avoid SettingWithCopyWarning
    
    # Ensure all columns in correlation_with_target exist in the DataFrame
    cols_to_drop = [col for col in correaltion_with_target if col in data_copy.columns]
   
    
    # Drop the identified columns from the copy_data
    data_copy = data_copy.drop(columns=cols_to_drop, inplace=False)
    
    return data_copy 



#####################################################################################


class OutliersClamping(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        num_cols, _ = update(X)
        self.clamping_bounds = {}

        for col in num_cols:
            Q1 = np.percentile(X[col], q=25, method='midpoint')
            Q3 = np.percentile(X[col], q=75, method='midpoint')
            IQR = Q3 - Q1
            upper_bound = Q3 + (5 * IQR)
            lower_bound = Q1 - (5 * IQR)

            self.clamping_bounds[col] = {'lower_bound': lower_bound, 'upper_bound': upper_bound}

        return self

    def transform(self, X):
        data_copy = X.copy()

        for col, bounds in self.clamping_bounds.items():
            lower_bound = bounds['lower_bound']
            upper_bound = bounds['upper_bound']
            data_copy[col] = np.clip(data_copy[col], lower_bound, upper_bound)

        return data_copy

#####################################################################################



class SkewnessTransformer(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.skew_cols = []
    
    def fit(self, X, y=None):
        # Store the columns that need to be transformed
        self.skew_cols = self.get_skewness_cols(X)
        return self
    
    def transform(self, X):
        # Apply log transformation to the stored columns
        X_copy = X.copy()
        for col in self.skew_cols:
            X_copy[col] = np.log1p(np.abs(X_copy[col]))

        return X_copy
    
    def get_skewness_cols(self, data):
        num_cols, _ = update(data)
        skew_cols = []
        
        # Check skewness for numerical columns
        skewness = self.check_skewness(data[num_cols])
        for i, col in enumerate(num_cols):
            if np.abs(skewness[i]) > 1:
                skew_cols.append(col)
        
        return skew_cols
    
    def check_skewness(self, data):
        x_mean = data.mean(axis=0, numeric_only=True)
        std = data.std(axis=0)
        N = data.shape[0]

        skewness_coeff = (((data - x_mean) ** 3).sum(axis=0)) / ((N - 1) * std ** 3)
        return skewness_coeff

#####################################################################################

ordinal_cats = ['LotShape', 'ExterQual', 'BsmtQual', 'BsmtExposure', 'BsmtFinType1', 'HeatingQC',
                'KitchenQual', 'FireplaceQu','HasRemodeled','Has2ndFloor']


nominal_cats = ['MSZoning', 'LotConfig', 'Neighborhood', 'HouseStyle', 'RoofStyle', 'Exterior1st',
                'Exterior2nd', 'Foundation', 'GarageType', 'GarageFinish',
               'YrSold_cat','MoSold_cat','YearBuilt_cat','MSSubClass_cat']

class CustomLabelBinarizer(BaseEstimator, TransformerMixin):
    def __init__(self, columns=None):
        self.columns = columns
        self.label_encoders = {}
        self.label_binarizers = {}

    def fit(self, X, y=None):
        for column in self.columns:
            if column in ordinal_cats:
                le = LabelEncoder()
                le.fit(X[column])
                self.label_encoders[column] = le
            else:
                lb = LabelBinarizer()
                lb.fit(X[column])
                self.label_binarizers[column] = lb
        return self

    def transform(self, X):
        X_transformed = X.copy()
        for column in self.columns:
            if column in ordinal_cats:
                le = self.label_encoders[column]
                # Transform and handle unknown labels
                X_transformed[column] = le.transform(np.where(X_transformed[column].isin(le.classes_), X_transformed[column], le.classes_[0]))
            else:
                lb = self.label_binarizers[column]
                # Transform and handle unknown labels
                transformed_column = lb.transform(np.where(X_transformed[column].isin(lb.classes_), X_transformed[column], 'missing'))
                if lb.sparse_output:
                    transformed_column = pd.DataFrame.sparse.from_spmatrix(transformed_column,
                                                                          columns=[f"{column}_{i}" for i in range(transformed_column.shape[1])])
                else:
                    transformed_column = pd.DataFrame(transformed_column,
                                                       columns=[f"{column}_{i}" for i in range(transformed_column.shape[1])])

                X_transformed = pd.concat([X_transformed, transformed_column], axis=1)
                X_transformed = X_transformed.drop(column, axis=1)

        return X_transformed





#######################################################################


class CustomDataPreprocessor(BaseEstimator, TransformerMixin):
    def __init__(self, ordinal_cats, nominal_cats):
        self.ordinal_cats = ordinal_cats
        self.nominal_cats = nominal_cats
        self.ct = None  # Changed from self.column_transformer
        self.ohe_feature_names = None

    def fit(self, X, y=None):
        # Call the update function to get numerical columns
        num_cols, _ = update(X)

        # Create a label binarizer transformer
        label_binarizer_transformer = CustomLabelBinarizer(columns=self.ordinal_cats)
        ohe = OneHotEncoder(handle_unknown='ignore', sparse_output=False)

        # Create a column transformer
        self.ct = make_column_transformer(
            (MinMaxScaler(), num_cols),
            (ohe, self.nominal_cats),
            (label_binarizer_transformer, self.ordinal_cats),
            remainder='passthrough'
        )

        # Fit the transformers
        self.ct.fit(X)

        # Get the column names after the transformations
        self.ohe_feature_names = (
            list(self.ct
                 .named_transformers_['onehotencoder']
                 .get_feature_names_out(self.nominal_cats))
        )

        return self

    def transform(self, X):
        # Check if fit has been called
        if self.ct is None:
            raise ValueError("Must call fit before transforming data.")

        # Transform the data
        X_transformed = self.ct.transform(X)

        # Combine the transformed data with the original data
        columns = (
            list(self.ct.transformers_[0][2]) +
            list(self.ohe_feature_names) +
            list(self.ct.named_transformers_['customlabelbinarizer'].columns)
        )

        X_df = pd.DataFrame(X_transformed, columns=columns)

        return X_df
    
    
    

# make funciton transformer to correlation_with_target
drop_correlation_with_target_transformer = FunctionTransformer(drop_correlation_with_target , kw_args = {'correaltion_with_target': correaltion_with_target})


# Original pipeline
pipe = Pipeline([
    ('feature_engineering_transformer',feature_engineering_transformer),
    ('DropNullColumns', DropNullColumns()),
    ('Impute' , Impute()),
    ('DropColumnsWithSameValues' , DropColumnsWithSameValues()),
    ('drop_correaltion_with_target_transformer' , drop_correlation_with_target_transformer),
    ('OutliersClamping', OutliersClamping()),
    ('SkewnessTransformer', SkewnessTransformer()),
    ('preprocess', CustomDataPreprocessor(ordinal_cats=ordinal_cats, nominal_cats=nominal_cats))
])

# Apply the pipeline to your dataset
X = df.drop(['SalePrice'], axis=1)
y = np.log1p(df['SalePrice'])


X_test_ = pd.read_csv('/kaggle/input/a-advanced-regression-techniques/test.csv')


X_preprocessed  = pipe.fit_transform(X)
X_test_preprocessed  = pipe.transform(X_test_)

In [93]:
# Displaying the Shapes of Preprocessed Training and Testing Data
print(f'X_train Shape : {X_preprocessed.shape}')
print(f'X_test Shape : {X_test_preprocessed.shape}')

X_train Shape : (1460, 278)
X_test Shape : (1459, 278)


In [94]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV, KFold, cross_val_score

# Split the data into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_preprocessed, y, test_size=0.2, random_state=42)

# Define the models
models = {
    'LinearRegression': LinearRegression(),
    'RandomForest': RandomForestRegressor(random_state=42),
    'XGBoost': XGBRegressor(random_state=42)
}

# Define the hyperparameter grids for each model
param_grids = {
    'LinearRegression': {},
    'RandomForest': {
        'n_estimators': [100, 200, 500],
        'max_depth': [None, 10, 30],
        'min_samples_split': [2, 5, 10],
    },
    'XGBoost': {
        'n_estimators': [100, 200, 500],
        'learning_rate': [0.01, 0.1, 0.3],
        'max_depth': [3, 6, 10],
    }
}

# 3-fold cross-validation
cv = KFold(n_splits=3, shuffle=True, random_state=42)

# Train and tune the models
grids = {}
for model_name, model in models.items():
    #print(f'Training and tuning {model_name}...')
    grids[model_name] = GridSearchCV(estimator=model, param_grid=param_grids[model_name], cv=cv, scoring='neg_mean_squared_error', n_jobs=-1, verbose=2)
    grids[model_name].fit(X_train, y_train)
    best_params = grids[model_name].best_params_
    best_score = np.sqrt(-1 * grids[model_name].best_score_)
    
    print(f'Best parameters for {model_name}: {best_params}')
    print(f'Best RMSE for {model_name}: {best_score}\n')

Fitting 3 folds for each of 1 candidates, totalling 3 fits
Best parameters for LinearRegression: {}
Best RMSE for LinearRegression: 1854421721.5268085

Fitting 3 folds for each of 27 candidates, totalling 81 fits
Best parameters for RandomForest: {'max_depth': None, 'min_samples_split': 2, 'n_estimators': 100}
Best RMSE for RandomForest: 0.1497519538814267

Fitting 3 folds for each of 27 candidates, totalling 81 fits
Best parameters for XGBoost: {'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 200}
Best RMSE for XGBoost: 0.1401222146221897



### the results with metric score `r2`

- Fitting 3 folds for each of 1 candidates, totalling 3 fits
- Best parameters for LinearRegression: {}
- Best r2 validation score for LinearRegression: `-4.663684075662625e+18``



- Fitting 3 folds for each of 27 candidates, totalling 81 fits
- Best parameters for RandomForest: `{'max_depth': None, 'min_samples_split': 2, 'n_estimators': 500}`
- Best r2 validation score for RandomForest: `0.8656334692119465`



- Fitting 3 folds for each of 27 candidates, totalling 81 fits
- Best parameters for XGBoost: `{'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 100}`
- Best r2 validation score for XGBoost: `0.8781604167887959`

### the results with metric score `neg_mean_squared_error`


- Fitting 3 folds for each of 1 candidates, totalling 3 fits
- Best parameters for LinearRegression: {}
- Best RMSE for LinearRegression: `1854421721.5268085`



- Fitting 3 folds for each of 27 candidates, totalling 81 fits
- Best parameters for RandomForest: {'max_depth': None, 'min_samples_split':  2,'n_estimators': 100}
- Best RMSE for RandomForest:` 0.1497519538814267`



- Fitting 3 folds for each of 27 candidates, totalling 81 fits
- Best parameters for XGBoost: {'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 200}
- Best RMSE for XGBoost: `0.1401222146221897`

In [95]:

from sklearn.neural_network import MLPRegressor

X_train_scaled = X_train.copy()
X_test_scaled = X_test.copy()

# Create an MLPRegressor instance
mlp = MLPRegressor(random_state=42,max_iter=10000, n_iter_no_change = 3,learning_rate_init=0.001)

# Define the parameter grid for tuning
param_grid = {
    'hidden_layer_sizes': [(10,), (10,10), (10,10,10), (25)],
    'activation': ['relu', 'tanh'],
    'solver': ['adam'],
    'alpha': [0.0001, 0.001, 0.01],
    'learning_rate': ['constant', 'invscaling', 'adaptive'],
}

# Create the GridSearchCV object
grid_search_mlp = GridSearchCV(mlp, param_grid, scoring='neg_mean_squared_error', cv=3, n_jobs=-1, verbose=1)

# Fit the model on the training data
grid_search_mlp.fit(X_train_scaled, y_train)

# Print the best parameters found during the search
print("Best parameters found: ", grid_search_mlp.best_params_)

# Evaluate the model on the test data
best_score = np.sqrt(-1 * grid_search_mlp.best_score_)
print("Test score: ", best_score)


Fitting 3 folds for each of 72 candidates, totalling 216 fits
Best parameters found:  {'activation': 'relu', 'alpha': 0.001, 'hidden_layer_sizes': (10,), 'learning_rate': 'constant', 'solver': 'adam'}
Test score:  0.3208466710435529


## Having cleaned our training data from multi-collinearity, we can measure the significance level of each feature and perform subset selection using the backward-stepwise method


In [96]:
from statsmodels.api import OLS

# Fit OLS regression model and get summary
LRresult = OLS(y_train, X_train).fit().summary2().tables[1]

# Reset index and rename the column to 'feature_name'
LRresult = LRresult.reset_index().rename(columns={'index': 'feature_name'})

In [97]:
# order the columns for the most important feature (eigen vector) to the least
LRresult.sort_values(by = 'Coef.' , ascending = False)

,feature_name,Coef.,Std.Err.,t,P>|t|,[0.025,0.975]
6,YearBuilt,2.048437,0.037744,54.271546,1.084134e-287,1.974362,2.122513
24,PropertyAge,1.733692,0.040718,42.578538,6.901812e-219,1.653781,1.813603
25,TotalSF,1.539231,0.292280,5.266283,1.737605e-07,0.965610,2.112852
124,YrSold_cat_2006,1.376083,0.016272,84.566736,0.000000e+00,1.344148,1.408018
23,YrSold,1.152308,0.013307,86.594848,0.000000e+00,1.126192,1.178424
...,...,...,...,...,...,...,...
265,MSSubClass_cat_160,-0.157400,0.044056,-3.572737,3.717875e-04,-0.243863,-0.070937
266,MSSubClass_cat_180,-0.158845,0.062255,-2.551518,1.088782e-02,-0.281026,-0.036665
162,YearBuilt_cat_1917,-0.208802,0.135942,-1.535963,1.248946e-01,-0.475598,0.057994
149,YearBuilt_cat_1898,-0.311926,0.138180,-2.257400,2.421917e-02,-0.583114,-0.040739


In [98]:
def backward_stepwise_selection(y, X):
    """
    Performs backward stepwise selection to drop insignificant features based on p-values obtained from OLS regression.

    Parameters:
    - y: Target variable (dependent variable).
    - X_vif: DataFrame containing independent variables.

    Returns:
    - X_vif_copy: DataFrame with insignificant features dropped.
    - insignificant_feats_ls: List of dropped insignificant features.
    """

    # Make a copy of the input DataFrame
    X_copy = X.copy()

    # List to store names of insignificant features
    insignificant_feats_ls = []

    # Fit OLS regression model and get summary
    LR_result = OLS(y, X_copy).fit().summary2().tables[1].reset_index().rename(columns={'index': 'feature_name'})

    # Loop until all features are significant
    while LR_result['P>|t|'].max() > 0.05:
        # Find the feature with the maximum p-value
        insignificant_feat = LR_result.loc[LR_result['P>|t|'].idxmax(), 'feature_name']
        # Append the insignificant feature to the list
        insignificant_feats_ls.append(insignificant_feat)
        # Drop the insignificant feature from the copy df
        X_copy = X_copy.drop(columns=[insignificant_feat])
        # Recalculate p-values after dropping the feature
        LR_result = OLS(y, X_copy).fit().summary2().tables[1].reset_index().rename(columns={'index': 'feature_name'})

    return insignificant_feats_ls  , X_copy



# Call the function
dropped_features , X_backward  = backward_stepwise_selection(y_train, X_train)

In [99]:
len(dropped_features)

166

In [100]:
X_backward.shape

(1168, 112)

In [101]:
# Fit OLS regression model and get summary
LRresult = OLS(y_train, X_backward).fit().summary2().tables[0]

In [102]:
LRresult

,0,1,2,3
0,Model:,OLS,Adj. R-squared:,0.910
1,Dependent Variable:,SalePrice,AIC:,-1601.3240
2,Date:,2024-01-10 20:14,BIC:,-1100.0822
3,No. Observations:,1168,Log-Likelihood:,899.66
4,Df Model:,98,F-statistic:,121.6
5,Df Residuals:,1069,Prob (F-statistic):,0.00
6,R-squared:,0.918,Scale:,0.013707


In [103]:
def drop_cols_comes_from_backward(data ,dropped_features ):
    data_copy = data.copy()  # Create a copy to avoid SettingWithCopyWarning
    
    # Ensure all columns in cols_of_null exist in the DataFrame
    cols_to_drop = [col for col in dropped_features if col in data_copy.columns]
    
    # Drop the identified columns from the copy_data
    data_copy = data_copy.drop(columns=cols_to_drop, inplace=False)
    
    return data_copy

drop_cols_comes_from_backward_transformer = FunctionTransformer(drop_cols_comes_from_backward , kw_args = {'dropped_features': dropped_features})
pipe_backward = Pipeline([
    ('preprocessing', pipe),
    ('drop_cols_comes_from_backward_transformer',drop_cols_comes_from_backward_transformer)
])

X_backward = pipe_backward[-1].fit_transform(X_preprocessed)
X_test_backward = pipe_backward[-1].transform(X_test_preprocessed)

In [104]:
print(f'X_train Shape : {X_backward.shape}')
print(f'X_test Shape : {X_test_backward.shape}')

X_train Shape : (1460, 112)
X_test Shape : (1459, 112)


In [106]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV, KFold, cross_val_score

# Split the data into training and testing sets
from sklearn.model_selection import train_test_split
X_train_backward, X_test_backward, y_train_backward, y_test_backward = train_test_split(X_backward, y, test_size=0.2, random_state=42)

# Define the models
models = {
    'Lasso': Lasso(),
    'RandomForest': RandomForestRegressor(random_state=42),
    'XGBoost': XGBRegressor(random_state=42)
}

# Define the hyperparameter grids for each model
param_grids = {
    'Lasso': {'alpha' : [0.1 ,0.01 ,0.001]},
    'RandomForest': {
        'n_estimators': [100, 200, 500],
        'max_depth': [None, 10, 30],
        'min_samples_split': [2, 5, 10],
    },
    'XGBoost': {
        'n_estimators': [100, 200, 500],
        'learning_rate': [0.01, 0.1, 0.3],
        'max_depth': [3, 6, 10],
    }
}

# 3-fold cross-validation
cv = KFold(n_splits=3, shuffle=True, random_state=42)

# Train and tune the models
grids_backward = {}
for model_name, model in models.items():
    #print(f'Training and tuning {model_name}...')
    grids_backward[model_name] = GridSearchCV(estimator=model, param_grid=param_grids[model_name], cv=cv, scoring='neg_mean_squared_error', n_jobs=-1, verbose=2)
    grids_backward[model_name].fit(X_train_backward, y_train_backward)
    best_params = grids_backward[model_name].best_params_
    best_score = np.sqrt(-1 * grids_backward[model_name].best_score_)
    
    print(f'Best parameters for {model_name}: {best_params}')
    print(f'Best RMSE for {model_name}: {best_score}\n')

Fitting 3 folds for each of 3 candidates, totalling 9 fits
Best parameters for Lasso: {'alpha': 0.001}
Best RMSE for Lasso: 0.13262386167668314

Fitting 3 folds for each of 27 candidates, totalling 81 fits
Best parameters for RandomForest: {'max_depth': None, 'min_samples_split': 2, 'n_estimators': 500}
Best RMSE for RandomForest: 0.14673813253235998

Fitting 3 folds for each of 27 candidates, totalling 81 fits
Best parameters for XGBoost: {'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 100}
Best RMSE for XGBoost: 0.13889639133256979



## the results with metric score `r2`:

- Fitting 3 folds for each of 1 candidates, totalling 3 fits
- Best parameters for Lasso: {}
- Best **r2 validation score for Lasso**: `0.88546`



- Fitting 3 folds for each of 27 candidates, totalling 81 fits
- Best parameters for RandomForest: `{'max_depth': None, 'min_samples_split': 2, 'n_estimators': 500}`
- Best **r2 validation score for RandomForest**:`0.8586784735347418`



- Fitting 3 folds for each of 27 candidates, totalling 81 fits
- Best parameters for XGBoost: `{'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 100}`
- Best **r2 validation score for XGBoost**: `0.8736290820145624`




### the results with metric score `neg_mean_squared_error`:

- Fitting 3 folds for each of 1 candidates, totalling 3 fits
- Best parameters for Lasso: {0.001}
- Best RMSE for Lasso: `0.13262386167668314`



- Fitting 3 folds for each of 27 candidates, totalling 81 fits
- Best parameters for RandomForest: `{'max_depth': None, 'min_samples_split': 2, 'n_estimators': 500}`
- Best RMSE for RandomForest: `0.14673813253235998
`



- Fitting 3 folds for each of 27 candidates, totalling 81 fits
- Best parameters for XGBoost: `{'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 100}`
- Best RMSE for XGBoost: `0.13889639133256979`

In [107]:
from sklearn.neural_network import MLPRegressor

X_train_scaled_backward = X_train_backward.copy()
X_test_scaled_backward = X_test_backward.copy()

# Create an MLPRegressor instance
mlp = MLPRegressor(random_state=42,max_iter=10000, n_iter_no_change = 3,learning_rate_init=0.001)

# Define the parameter grid for tuning
param_grid = {
    'hidden_layer_sizes': [(10,), (10,10), (10,10,10), (25)],
    'activation': ['relu', 'tanh'],
    'solver': ['adam'],
    'alpha': [0.0001, 0.001, 0.01, .1, 1],
    'learning_rate': ['constant', 'invscaling', 'adaptive'],
}

# Create the GridSearchCV object
grid_search_mlp_backward = GridSearchCV(mlp, param_grid, scoring='neg_mean_squared_error', cv=3, n_jobs=-1, verbose=1)

# Fit the model on the training data
grid_search_mlp_backward.fit(X_train_scaled_backward, y_train_backward)

# Print the best parameters found during the search
print("Best parameters found: ", grid_search_mlp_backward.best_params_)

# Evaluate the model on the test data
best_score = np.sqrt(-1 * grid_search_mlp_backward.best_score_)
print("Test score: ", best_score)

Fitting 3 folds for each of 120 candidates, totalling 360 fits
Best parameters found:  {'activation': 'relu', 'alpha': 1, 'hidden_layer_sizes': 25, 'learning_rate': 'constant', 'solver': 'adam'}
Test score:  0.167260255824758


In [108]:

xgb = XGBRegressor(learning_rate=0.1, max_depth=3, n_estimators=100)

xgb.fit(X_train_backward , y_train_backward)


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=3, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=100, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [109]:
# the score for validation data 
xgb.score(X_test_backward,y_test_backward)

0.9000128380116853

In [110]:
y_hat_sklearn = xgb.predict(X_train_backward)

### Checking the linear regression model assumptions
#### the epslon term (Erorr) 'Resudelas' is assumed to be

- Has mean of zero
- Is normally dittributed
- Has constant variance at evrey value of X (not fanning out)

In [111]:
# Calculate residuals
residuals = y_train - y_hat_sklearn

# Scatter plot of residuals against predicted values using Plotly Express
fig = px.scatter(x=y_hat_sklearn, y=residuals, title='Residuals vs. Predicted Values: Checking Linear Regression Model Assumptions',
                 labels={'x': 'Predicted Values', 'y': 'Residuals'},
                 template='plotly_dark', opacity=0.6, color_discrete_sequence=['blue'])

# Update layout for better visibility
fig.update_layout(
    xaxis=dict(title='Predicted Values'),
    yaxis=dict(title='Residuals'),
    showlegend=False
)

# Display the Plotly figure
fig.show()


In [112]:
predicted = xgb.predict(X_test_backward)

In [113]:
# Scatter plot of actual vs predicted values using Plotly Express
fig17 = px.scatter(x=y_test_backward, y=predicted, title='Actual vs Predicted Prices',
                 labels={'x': 'Actual Prices', 'y': 'Predicted Prices'},
                 template='plotly_dark', opacity=0.5, color_discrete_sequence=['blue'])

# Add a regression line to the scatter plot using Graph Objects
regression_line = go.Scatter(x=[min(y_test), max(y_test)], y=[min(y_test), max(y_test)],
                             mode='lines', line=dict(color='red', dash='dash'),
                             name='Regression Line')

fig17.add_trace(regression_line)

# Update layout for better visibility
fig17.update_layout(
    xaxis=dict(title='Actual Prices'),
    yaxis=dict(title='Predicted Prices'),
)

# Display the Plotly figure
fig17.show()

# Principal Component Analysis 
- Basic Feature Engineering 

In [114]:
#pca
from sklearn.decomposition import PCA

pca = PCA()
X_pca_pre = pca.fit_transform(X_preprocessed)

# Calculate the cumulative explained variance
cumulative_explained_variance = np.cumsum(pca.explained_variance_ratio_)

# Choose the number of components based on the explained variance threshold
n_components = np.argmax(cumulative_explained_variance >= 0.95) + 1

pca = PCA(n_components=n_components)
pipe_pca = Pipeline(steps=
                        [('preprocessor', pipe),
                        ('pca', pca)])

X_pca = pipe_pca[-1].fit_transform(X_preprocessed)
X_test_pca = pipe_pca[-1].transform(X_test_preprocessed)

In [115]:
print(f'X_train Shape : {X_pca.shape}')
print(f'X_test Shape : {X_test_pca.shape}')

X_train Shape : (1460, 77)
X_test Shape : (1459, 77)


In [116]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV, KFold, cross_val_score

# Split the data into training and testing sets
from sklearn.model_selection import train_test_split
X_train_pca, X_test_pca, y_train_pca, y_test_pca = train_test_split(X_pca, y, test_size=0.2, random_state=42)

# Define the models
models = {
    'LinearRegression': LinearRegression(),
    'RandomForest': RandomForestRegressor(random_state=42),
    'XGBoost': XGBRegressor(random_state=42)
}

# Define the hyperparameter grids for each model
param_grids = {
    'LinearRegression': {},
    'RandomForest': {
        'n_estimators': [100, 200, 500],
        'max_depth': [None, 10, 30],
        'min_samples_split': [2, 5, 10],
    },
    'XGBoost': {
        'n_estimators': [100, 200, 500],
        'learning_rate': [0.01, 0.1, 0.3],
        'max_depth': [3, 6, 10],
    }
}

# 3-fold cross-validation
cv = KFold(n_splits=3, shuffle=True, random_state=42)

# Train and tune the models
grids_pca = {}
for model_name, model in models.items():
    #print(f'Training and tuning {model_name}...')
    grids_pca[model_name] = GridSearchCV(estimator=model, param_grid=param_grids[model_name], cv=cv, scoring='neg_mean_squared_error', n_jobs=-1, verbose=2)
    grids_pca[model_name].fit(X_train_pca, y_train_pca)
    best_params = grids_pca[model_name].best_params_
    best_score = np.sqrt(-1 * grids_pca[model_name].best_score_)
    
    print(f'Best parameters for {model_name}: {best_params}')
    print(f'Best RMSE for {model_name}: {best_score}\n')

Fitting 3 folds for each of 1 candidates, totalling 3 fits
Best parameters for LinearRegression: {}
Best RMSE for LinearRegression: 0.15581845638624992

Fitting 3 folds for each of 27 candidates, totalling 81 fits
Best parameters for RandomForest: {'max_depth': None, 'min_samples_split': 2, 'n_estimators': 500}
Best RMSE for RandomForest: 0.21894584971153208

Fitting 3 folds for each of 27 candidates, totalling 81 fits
Best parameters for XGBoost: {'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 500}
Best RMSE for XGBoost: 0.1900598737795888



## the results with metric score `r2` and 120 Component:

- Fitting 3 folds for each of 1 candidates, totalling 3 fits
- Best parameters for LinearRegression: {}
- Best **r2 validation score for LinearRegression and 77 component**: `0.8506764843267046`
- Best **r2 validation score for LinearRegression and 120 component**: `0.8427361517031544`



- Fitting 3 folds for each of 27 candidates, totalling 81 fits
- Best parameters for RandomForest: `{'max_depth': None, 'min_samples_split': 2, 'n_estimators': 500}`
- Best **r2 validation score for RandomForest**:`0.6772771424989977`



- Fitting 3 folds for each of 27 candidates, totalling 81 fits
- Best parameters for XGBoost: `{'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 500}`
- Best **r2 validation score for XGBoost**: `0.7626958028442977`




### the results with metric score `neg_mean_squared_error`:

- Fitting 3 folds for each of 1 candidates, totalling 3 fits
- Best parameters for LinearRegression: {}
- Best RMSE for LinearRegression: `0.15481074226634664`



- Fitting 3 folds for each of 27 candidates, totalling 81 fits
- Best parameters for RandomForest: `{'max_depth': None, 'min_samples_split': 2, 'n_estimators': 200}`
- Best RMSE for RandomForest: `0.21823518494880578`



- Fitting 3 folds for each of 27 candidates, totalling 81 fits
- Best parameters for XGBoost: `{'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 500}`
- Best RMSE for XGBoost: `0.18907789357085342`

### MSE for our Validation data

In [117]:
# Importing necessary library
from sklearn.metrics import mean_squared_error

# Looping through different models in a dictionary (grids)
for model_name, model in grids.items():
    # Predicting the target variable using the current model
    predictions = model.predict(X_test)
    
    # Calculating the mean squared error (MSE) between the predicted and actual values
    mse_value = mean_squared_error(predictions, y_test)
    
    # Printing the model name and its corresponding MSE
    print(f"{model_name}: {mse_value:.2f}")

# Note: The code evaluates and prints the mean squared error for each model in the 'grids' dictionary


LinearRegression: 76134169817974928.00
RandomForest: 0.02
XGBoost: 0.02


In [118]:
# Looping through different models in a dictionary (grids_backward)
for model_name, model in grids_backward.items():
    # Predicting the target variable using the current model with Backward-transformed test data
    predictions = model.predict(X_test_backward)
    
    # Calculating the mean squared error (MSE) between the predicted and actual values
    mse_value = mean_squared_error(predictions, y_test_backward)
    
    # Printing the model name and its corresponding MSE
    print(f"{model_name}: {mse_value:.2f}")

# Note: The code evaluates and prints the mean squared error for each model in the 'grids_backward' dictionary,
# using Backward-transformed test data

Lasso: 0.02
RandomForest: 0.02
XGBoost: 0.02


In [119]:
# Looping through different models in a dictionary (grids_pca)
for model_name, model in grids_pca.items():
    # Predicting the target variable using the current model with PCA-transformed test data
    predictions = model.predict(X_test_pca)
    
    # Calculating the mean squared error (MSE) between the predicted and actual values
    mse_value = mean_squared_error(predictions, y_test)
    
    # Printing the model name and its corresponding MSE
    print(f"{model_name}: {mse_value:.2f}")

# Note: The code evaluates and prints the mean squared error for each model in the 'grids_pca' dictionary,
# using PCA-transformed test data


LinearRegression: 0.03
RandomForest: 0.05
XGBoost: 0.04


In [120]:
print( str(np.sqrt(mean_squared_error(grid_search_mlp.predict(X_test_scaled),y_test))))

0.22984311270518684


In [121]:
print( str(np.sqrt(mean_squared_error(grid_search_mlp_backward.predict(X_test_scaled_backward),y_test_backward))))

0.13734350786809865


In [122]:
param_grid = {
    'model__alpha': [0.001, 0.01, 0.1]
}

pipe_model = Pipeline([
    ('pipeline',pipe),
    ('model',Lasso())
])
# find the best set of parameters
grid = GridSearchCV(pipe_model , param_grid , cv=5, scoring='r2')
grid.fit(X, y)

# Print the best parameters and best score
print("Best Parameters:", grid.best_params_)
print("Best Cross-Validated Accuracy:", grid.best_score_)

Best Parameters: {'model__alpha': 0.001}
Best Cross-Validated Accuracy: 0.8947493891152579


## the results with metric score `r2`

- Best Parameters: `{'model__alpha': 0.001}`
- Best Cross-Validated Accuracy: `0.8947493891152579`

In [123]:
df_test_preprocessed = pipe_backward.transform(X_test_)

In [124]:
#xgboost submission
y_xgboost = np.exp(grids_backward['XGBoost'].predict(df_test_preprocessed))

df_xgboost_out = X_test_[['Id']].copy()
df_xgboost_out['SalePrice'] = y_xgboost

#
df_xgboost_out.to_csv('submission_xgboost_new_features_normalized.csv', index=False)

In [125]:
#rf submission
y_rf = np.exp(grids_backward['RandomForest'].predict(df_test_preprocessed))

df_rf_out = X_test_[['Id']].copy()
df_rf_out['SalePrice'] = y_rf

#
df_rf_out.to_csv('submission_rf_normalized.csv', index=False)

In [126]:
#mlp submission
y_mlp = np.exp(grid_search_mlp_backward.predict(df_test_preprocessed))

df_mlp_out = X_test_[['Id']].copy()
df_mlp_out['SalePrice'] = y_mlp

df_mlp_out.to_csv('submission_mlp_normalized.csv', index=False)

In [127]:
y_avg_ens = (y_rf + y_xgboost + y_mlp)/3

#xgboost submission
df_avg_ens_out = X_test_[['Id']].copy()
df_avg_ens_out['SalePrice'] = y_avg_ens

#
df_avg_ens_out.to_csv('submission_avg_ens_new_features_normalized.csv', index=False)


In [129]:
from sklearn.ensemble import StackingRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import LinearRegression
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.metrics import mean_squared_error
import numpy as np


grids_backward['MLP'] = grid_search_mlp_backward

best_estimators = [(model_name, grid.best_estimator_) for model_name, grid in grids_backward.items()]

# Define the candidate meta-models
meta_models = {
    'MLP': MLPRegressor(random_state=42, max_iter=20000, n_iter_no_change=10, learning_rate_init=0.001, early_stopping=True),
    'Lasso': Lasso(alpha = 0.001),
    'XGBoost': XGBRegressor(random_state=42)
}

# Define the hyperparameter grids for each meta-model
meta_param_grids = {
    'MLP': {
        'final_estimator__hidden_layer_sizes': [(10,)],
        'final_estimator__activation': ['relu'],
        'final_estimator__solver': ['sgd'],
        'final_estimator__alpha': [0.5],
        'final_estimator__learning_rate': ['constant'],
    },
    'Lasso': {},
    'XGBoost': {
        'final_estimator__n_estimators': [100, 200, 500],
        'final_estimator__learning_rate': [0.01, 0.1, 0.3],
        'final_estimator__max_depth': [3, 6, 10],
    }
}

# 3-fold cross-validation
cv = KFold(n_splits=3, shuffle=True, random_state=42)

# Train and tune the stacking ensemble
best_score = float('inf')
best_model = None

for meta_name, meta_model in meta_models.items():
    print(f'Training and tuning {meta_name} as the meta-model...')
    
    # Stacking Regressor with early stopping for MLP
    if meta_name == 'MLP':
        stacking_regressor = StackingRegressor(estimators=best_estimators, final_estimator=meta_model, cv=cv, n_jobs=-1)
    else:
        stacking_regressor = StackingRegressor(estimators=best_estimators, final_estimator=meta_model, cv=cv)
    
    grid_search = GridSearchCV(estimator=stacking_regressor, param_grid=meta_param_grids[meta_name], cv=cv, scoring='neg_mean_squared_error', n_jobs=-1, verbose=1)
    grid_search.fit(X_train_backward, y_train_backward)
    
    best_params = grid_search.best_params_
    best_rmse = np.sqrt(-1 * grid_search.best_score_)

    print(f'Best parameters for {meta_name}: {best_params}')
    print(f'Best RMSE for {meta_name}: {best_rmse}\n')

    if best_rmse < best_score:
        best_score = best_rmse
        best_model = grid_search

# Evaluate the best stacking ensemble on the test data
y_pred = best_model.predict(X_test_backward)
rmse = np.sqrt(mean_squared_error(y_test_backward, y_pred))
print(f"Best stacking ensemble's RMSE on test data: {rmse}")


Training and tuning MLP as the meta-model...
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Best parameters for MLP: {'final_estimator__activation': 'relu', 'final_estimator__alpha': 0.5, 'final_estimator__hidden_layer_sizes': (10,), 'final_estimator__learning_rate': 'constant', 'final_estimator__solver': 'sgd'}
Best RMSE for MLP: 0.13169345700360363

Training and tuning Lasso as the meta-model...
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Best parameters for Lasso: {}
Best RMSE for Lasso: 0.12966112670675436

Training and tuning XGBoost as the meta-model...
Fitting 3 folds for each of 27 candidates, totalling 81 fits
Best parameters for XGBoost: {'final_estimator__learning_rate': 0.01, 'final_estimator__max_depth': 3, 'final_estimator__n_estimators': 500}
Best RMSE for XGBoost: 0.13283376095690286

Best stacking ensemble's RMSE on test data: 0.13160879205965167


In [130]:
y_stack = np.exp(best_model.predict(df_test_preprocessed))

#xgboost submission

df_stack_out = X_test_[['Id']].copy()
df_stack_out['SalePrice'] = y_stack

df_stack_out.to_csv('submission_stack_new_features_normalized.csv', index=False)

In [131]:
df_stack_out.to_csv('submission.csv', index=False)